# Neural Network

In [11]:
import pickle
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Import Data

In [2]:
# import data
all_tracks = pickle.load(open('all_tracks.pkl', 'rb'))
raag = pickle.load(open('raag.pkl', 'rb'))

# Label Encode Raags

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(raag)

LabelEncoder()

In [4]:
le.classes_

array(['Asa', 'Asa Kafi', 'Asavari', 'Bairarri', 'Basant', 'Dhanasri'],
      dtype='<U8')

In [5]:
y = le.transform(raag)
y

array([0, 0, 0, ..., 5, 5, 5])

In [48]:
# change it back
#le.inverse_transform

# Split Data

In [6]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(all_tracks), 
                                                    np.array(y),
                                                    test_size=0.33,
                                                    random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                                y_test,
                                                test_size=0.5,
                                                random_state=42)


# CNN

In [7]:
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]


In [8]:
def build_model(input_shape, output_layers):
    
    # create model
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten the output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer 
    model.add(keras.layers.Dense(output_layers, activation='softmax'))

    return model


In [9]:
X_train.shape

(4543, 128, 196, 1)

In [10]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape, 6)

In [12]:
# compile network

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [14]:
# train model

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    batch_size=32,
                    epochs=30) # refine

Epoch 1/30
142/142 [==============================] - 81s 562ms/step - loss: 1.4645 - accuracy: 0.4658 - val_loss: 1.3423 - val_accuracy: 0.4665
Epoch 2/30
142/142 [==============================] - 75s 524ms/step - loss: 1.1323 - accuracy: 0.5661 - val_loss: 1.1883 - val_accuracy: 0.5594
Epoch 3/30
142/142 [==============================] - 71s 503ms/step - loss: 0.9609 - accuracy: 0.6381 - val_loss: 1.1729 - val_accuracy: 0.5416
Epoch 4/30
142/142 [==============================] - 75s 529ms/step - loss: 0.8004 - accuracy: 0.7050 - val_loss: 0.9781 - val_accuracy: 0.6309
Epoch 5/30
142/142 [==============================] - 76s 536ms/step - loss: 0.6660 - accuracy: 0.7587 - val_loss: 0.9213 - val_accuracy: 0.6399
Epoch 6/30
142/142 [==============================] - 75s 530ms/step - loss: 0.5565 - accuracy: 0.8034 - val_loss: 0.7784 - val_accuracy: 0.7230
Epoch 7/30
142/142 [==============================] - 78s 549ms/step - loss: 0.4836 - accuracy: 0.8369 - val_loss: 0.6441 - val_ac

In [16]:
# evaluate model
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test set accuracy: {}".format(test_accuracy))

35/35 [==============================] - 3s 75ms/step - loss: 0.3568 - accuracy: 0.8874
Test set accuracy: 0.887399435043335


# Predict

In [20]:
def predict(model, X, y):

    X = X[np.newaxis, ...] # make into a 4d array
    prediction = model.predict(X)

    # extract index with max value (most likely raag)
    prediced_index = np.argmax(prediction, axis=1) # returns a 1d array
    print("Expected index: {}, Predicted index {}".format(y, prediced_index))


In [ ]:
# prediction on a sample
X_pred = X_test[77]
y_pred = y_test[77]

predict(model, X_pred, y_pred)